In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import json
import ipywidgets as widgets
from ipywidgets import interact, VBox, HBox, Output
from pathlib import Path
from IPython.display import display


In [2]:
%matplotlib inline

In [3]:
SCREEN_SIZE = (1024, 768)  # (width, height)

In [ ]:
def plot_fixations_single_image(
    fix_dict: dict,
    image_folder: str,
    image_key: str = "first_image",
    screen_size=(1024, 768),
    color_by: str = "order",
    point_size: float = 50,
    connect_path: bool = True,
    title: str | None = None,
    show_outline: bool = False,
    ax=None
):
    """
    Display stimulus image and overlay EyeLink fixations.
    Can be used standalone or with a provided axes object.
    """
    img_name = fix_dict.get(image_key)
    if not img_name:
        raise ValueError(f"No '{image_key}' field found in this trial.")
    image_path = Path(image_folder) / img_name

    if not image_path.exists():
        raise FileNotFoundError(f"Image not found: {image_path}")

    img = Image.open(image_path).convert("RGB")
    screen_w, screen_h = screen_size

    l, t, r, b = (screen_w//2 - 310, screen_h//2 - 310, 
                  screen_w//2 + 310, screen_h//2 + 310)

    xs = np.asarray(fix_dict["fix_x"], dtype=float)
    ys = np.asarray(fix_dict["fix_y"], dtype=float)
    durs = np.asarray(fix_dict.get("fix_dur_ms", [np.nan] * len(xs)), dtype=float)
    order = np.asarray(fix_dict.get("fix_index", np.arange(1, len(xs) + 1)), dtype=float)

    mask = (xs >= l) & (xs <= r) & (ys >= t) & (ys <= b)
    xs, ys, durs, order = xs[mask], ys[mask], durs[mask], order[mask]
    
    if xs.size == 0:
        print(f"⚠️ No fixations found for {image_key}")
        return None

    if ax is None:
        fig, ax = plt.subplots(figsize=(screen_w / 128, screen_h / 128), dpi=128)
        standalone = True
    else:
        standalone = False

    ax.imshow(img, extent=(l, r, b, t))

    if show_outline:
        ax.add_patch(plt.Rectangle(
            (l, t), r-l, b-t,
            fill=False, color='cyan', lw=1.5, linestyle='--'
        ))

    if color_by.lower() in ("duration", "dur", "fix_dur_ms"):
        cvals, clabel, cmap = durs, "Duration (ms)", "viridis"
    else:
        cvals, clabel, cmap = order, "Fixation order", "plasma"

    sc = ax.scatter(xs, ys, s=point_size, c=cvals, cmap=cmap,
                    edgecolor="white", linewidth=0.5, alpha=0.9)
    
    if connect_path and xs.size > 1:
        ax.plot(xs, ys, lw=1.5, alpha=0.7, color="white")

    if standalone:
        cb = plt.colorbar(sc, ax=ax, fraction=0.046, pad=0.04)
        cb.set_label(clabel)

    ax.set_xlim(0, screen_w)
    ax.set_ylim(screen_h, 0)
    ax.set_xlabel("x (screen px)")
    ax.set_ylabel("y (screen px)")
    ax.set_title(title or f"{image_key}")

    if standalone:
        plt.tight_layout()
        plt.show()
    
    return sc

In [ ]:
def plot_fixations_dual_images(
    fix_dict: dict,
    image_folder: str,
    screen_size=(1024, 768),
    color_by: str = "order",
    point_size: float = 50,
    connect_path: bool = True,
    show_outline: bool = False,
):
    """
    Plot both first_image and second_image side by side with fixations.
    """
    subject_id = fix_dict.get('subject_id', '?')
    trial_idx = fix_dict.get('trial_index', '?')
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    
    sc1 = plot_fixations_single_image(
        fix_dict, image_folder, "first_image",
        screen_size, color_by, point_size, connect_path,
        f"First Image", show_outline, ax1
    )
    
    sc2 = plot_fixations_single_image(
        fix_dict, image_folder, "second_image",
        screen_size, color_by, point_size, connect_path,
        f"Second Image", show_outline, ax2
    )
    
    if sc1 is not None:
        fig.colorbar(sc1, ax=[ax1, ax2], fraction=0.046, pad=0.04)
    
    fig.suptitle(f"Subject {subject_id} • Trial {trial_idx}", fontsize=14, y=0.98)
    plt.tight_layout()
    plt.show()

In [ ]:
def plot_fixations_dual_images(
    fix_dict: dict,
    image_folder: str,
    screen_size=(1024, 768),
    color_by: str = "order",
    point_size: float = 50,
    connect_path: bool = True,
    show_outline: bool = False,
):
    """
    Plot both first_image and second_image side by side with fixations.
    """
    subject_id = fix_dict.get('subject_id', '?')
    trial_idx = fix_dict.get('trial_index', '?')
    
    fig = plt.figure(figsize=(15, 6))
    
    gs = fig.add_gridspec(1, 3, width_ratios=[1, 1, 0.05], wspace=0.3)
    ax1 = fig.add_subplot(gs[0, 0])
    ax2 = fig.add_subplot(gs[0, 1])
    cbar_ax = fig.add_subplot(gs[0, 2])
    
    sc1 = plot_fixations_single_image(
        fix_dict, image_folder, "first_image",
        screen_size, color_by, point_size, connect_path,
        f"First Image", show_outline, ax1
    )
    
    sc2 = plot_fixations_single_image(
        fix_dict, image_folder, "second_image",
        screen_size, color_by, point_size, connect_path,
        f"Second Image", show_outline, ax2
    )
    
    if sc1 is not None:
        if color_by.lower() in ("duration", "dur", "fix_dur_ms"):
            clabel = "Duration (ms)"
        else:
            clabel = "Fixation order"
        
        cb = fig.colorbar(sc1, cax=cbar_ax)
        cb.set_label(clabel, fontsize=10)
    
    fig.suptitle(f"Subject {subject_id} • Trial {trial_idx}", fontsize=14, y=0.98)
    plt.show()

In [ ]:
datasets = {
    "Training 1": {
        "json": "Training 1/training1.json",
        "images": "Training 1/training1_images",
        "dual_images": False
    },
    "Training 2": {
        "json": "Training 2/training2.json",
        "images": "Training 2/training2_images",
        "dual_images": True
    },
    "Testing": {
        "json": "Testing/testing.json",
        "images": "Testing/testing_images",
        "dual_images": True
    }
}

all_data = {}
for dataset_name, paths in datasets.items():
    json_path = Path(paths["json"])
    if json_path.exists():
        with open(json_path, "r", encoding="utf-8") as f:
            all_data[dataset_name] = {
                "data": json.load(f),
                "image_folder": Path(paths["images"]),
                "dual_images": paths["dual_images"]
            }
        print(f"✅ Loaded {dataset_name}: {len(all_data[dataset_name]['data'])} trials")
    else:
        print(f"⚠️ {dataset_name} not found at {json_path}")


✅ Loaded Training 1: 1364 trials
✅ Loaded Training 2: 2320 trials
✅ Loaded Testing: 2304 trials


In [ ]:
output = Output()

dataset_dropdown = widgets.Dropdown(
    options=list(all_data.keys()),
    description="Dataset:",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='250px')
)

subject_dropdown = widgets.Dropdown(
    description="Subject:",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='250px')
)

trial_dropdown = widgets.Dropdown(
    description="Trial:",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='550px')
)

color_dropdown = widgets.Dropdown(
    options=["order", "duration"],
    value="order",
    description="Color by:",
    layout=widgets.Layout(width='200px')
)

outline_checkbox = widgets.Checkbox(
    value=True,
    description="Show outline",
    layout=widgets.Layout(width='150px')
)

def update_subjects(*args):
    dataset_name = dataset_dropdown.value
    if dataset_name not in all_data:
        return
    
    data = all_data[dataset_name]["data"]
    subjects = sorted(set([trial["subject_id"] for trial in data]))
    subject_dropdown.options = subjects
    if subjects:
        subject_dropdown.value = subjects[0]

def update_trials(*args):
    dataset_name = dataset_dropdown.value
    subject_id = subject_dropdown.value
    
    if dataset_name not in all_data:
        return
    
    data = all_data[dataset_name]["data"]
    subject_trials = [t for t in data if t["subject_id"] == subject_id]
    
    trial_labels = []
    for i, t in enumerate(subject_trials):
        first_img = t.get('first_image', 'Unknown')
        if all_data[dataset_name]["dual_images"]:
            second_img = t.get('second_image', 'Unknown')
            label = f"{i+1:02d} — {first_img} | {second_img}"
        else:
            label = f"{i+1:02d} — {first_img}"
        trial_labels.append(label)
    
    trial_dropdown.options = trial_labels
    if trial_labels:
        trial_dropdown.value = trial_labels[0]
        show_trial()

def show_trial(*args):
    output.clear_output(wait=True)
    
    dataset_name = dataset_dropdown.value
    subject_id = subject_dropdown.value
    
    if dataset_name not in all_data or not trial_dropdown.value:
        with output:
            print("⚠️ No trial selected")
        return
    
    dataset_info = all_data[dataset_name]
    data = dataset_info["data"]
    image_folder = dataset_info["image_folder"]
    dual_images = dataset_info["dual_images"]
    
    subject_trials = [t for t in data if t["subject_id"] == subject_id]
    trial_labels = [trial_dropdown.options[i] for i in range(len(trial_dropdown.options))]
    idx = trial_labels.index(trial_dropdown.value)
    trial = subject_trials[idx]
    
    with output:
        print(f"📊 {dataset_name} • {subject_id} • Trial {idx+1}/{len(subject_trials)}")
        
        if dual_images:
            plot_fixations_dual_images(
                trial,
                image_folder=image_folder,
                color_by=color_dropdown.value,
                point_size=60,
                show_outline=outline_checkbox.value
            )
        else:
            plot_fixations_single_image(
                trial,
                image_folder=image_folder,
                image_key="first_image",
                color_by=color_dropdown.value,
                point_size=60,
                show_outline=outline_checkbox.value
            )

dataset_dropdown.observe(update_subjects, names="value")
subject_dropdown.observe(update_trials, names="value")
trial_dropdown.observe(show_trial, names="value")
color_dropdown.observe(show_trial, names="value")
outline_checkbox.observe(show_trial, names="value")

update_subjects()

display(VBox([
    HBox([dataset_dropdown, subject_dropdown]),
    trial_dropdown,
    HBox([color_dropdown, outline_checkbox]),
    output
]))